# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Nov  2 2022  1:34PM  249908        10   0086138902    ISDIN Corporation   
1  Nov  2 2022  1:34PM  249908        10   0086138915    ISDIN Corporation   
2  Nov  2 2022  1:34PM  249908        10   0086138996    ISDIN Corporation   
3  Nov  2 2022  1:34PM  249908        10   0086139548    ISDIN Corporation   
4  Nov  2 2022  1:34PM  249908        10   0086139681    ISDIN Corporation   
5  Nov  2 2022  1:34PM  249908        10   0086139690    ISDIN Corporation   
6  Nov  2 2022  1:34PM  249908        10   0086139691    ISDIN Corporation   
7  Nov  2 2022  1:34PM  249908        10   0086139709    ISDIN Corporation   
8  Nov  2 2022  1:31PM  249907        19  ADV80007274  AdvaGen Pharma, Ltd   
9  Nov  2 2022  1:31PM  249907        19  ADV80007275  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
42  249904       Released          9
43  249905       Released          1
44  249906       Released          1
45  249907       Released         12
46  249908       Released          8

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249904                NaN        NaN       9.0
249905                NaN        NaN       1.0
249906                NaN        NaN       1.0
249907                NaN        NaN      12.0
249908                NaN        NaN       8.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249828                0.0        0.0       1.0
249837                0.0        0.0       2.0
249846                3.0        9.0       3.0
249847                4.0        8.0       1.0
249855                0.0        3.0      10.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249828                  0          0         1
249837                  0          0         2
249846                  3          9         3
249847                  4          8         1
249855                  0          3        10

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249828          0          0         1
1               249837          0          0         2
2               249846          3          9         3
3               249847          4          8         1
4               249855          0          3        10

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               249828                             1
1               249837                             2
2               249846         3         9         3
3               249847         4         8         1
4               249855                   3        10

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Nov  2 2022  1:34PM  249908        10                ISDIN Corporation
8   Nov  2 2022  1:31PM  249907        19              AdvaGen Pharma, Ltd
20  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.
21  Nov  2 2022  1:18PM  249905        10                   ClearSpec, LLC
22  Nov  2 2022  1:16PM  249904        10                ISDIN Corporation
31  Nov  2 2022  1:04PM  249902        10                ISDIN Corporation
41  Nov  2 2022 12:52PM  249901        10                Yusen – 3D Matrix
42  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations
43  Nov  2 2022 12:47PM  249899        19             Emersa Waterbox, LLC
45  Nov  2 2022 12:45PM  249898        10                Yusen – 3D Matrix

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Nov  2 2022  1:34PM  249908        10                ISDIN Corporation   
1  Nov  2 2022  1:31PM  249907        19              AdvaGen Pharma, Ltd   
2  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.   
3  Nov  2 2022  1:18PM  249905        10                   ClearSpec, LLC   
4  Nov  2 2022  1:16PM  249904        10                ISDIN Corporation   
5  Nov  2 2022  1:04PM  249902        10                ISDIN Corporation   
6  Nov  2 2022 12:52PM  249901        10                Yusen – 3D Matrix   
7  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations   
8  Nov  2 2022 12:47PM  249899        19             Emersa Waterbox, LLC   
9  Nov  2 2022 12:45PM  249898        10                Yusen – 3D Matrix   

  Completed Executing  Released  
0                             8  
1                            12  
2                             1  
3                             1  
4                             9  
5                            10  
6                             1  
7                             1  
8                             2  
9                             1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Nov  2 2022  1:34PM  249908        10                ISDIN Corporation   
1  Nov  2 2022  1:31PM  249907        19              AdvaGen Pharma, Ltd   
2  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.   
3  Nov  2 2022  1:18PM  249905        10                   ClearSpec, LLC   
4  Nov  2 2022  1:16PM  249904        10                ISDIN Corporation   
5  Nov  2 2022  1:04PM  249902        10                ISDIN Corporation   
6  Nov  2 2022 12:52PM  249901        10                Yusen – 3D Matrix   
7  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations   
8  Nov  2 2022 12:47PM  249899        19             Emersa Waterbox, LLC   
9  Nov  2 2022 12:45PM  249898        10                Yusen – 3D Matrix   

   Released Executing Completed  
0         8                      
1        12                      
2         1                      
3         1                      
4         9                      
5        10                      
6         1                      
7         1                      
8         2                      
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Nov  2 2022  1:34PM  249908        10    ISDIN Corporation         8   
1  Nov  2 2022  1:31PM  249907        19  AdvaGen Pharma, Ltd        12   
2  Nov  2 2022  1:30PM  249906        21    NBTY Global, Inc.         1   
3  Nov  2 2022  1:18PM  249905        10       ClearSpec, LLC         1   
4  Nov  2 2022  1:16PM  249904        10    ISDIN Corporation         9   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Nov  2 2022  1:34PM  249908        10    ISDIN Corporation         8   
1  Nov  2 2022  1:31PM  249907        19  AdvaGen Pharma, Ltd        12   
2  Nov  2 2022  1:30PM  249906        21    NBTY Global, Inc.         1   
3  Nov  2 2022  1:18PM  249905        10       ClearSpec, LLC         1   
4  Nov  2 2022  1:16PM  249904        10    ISDIN Corporation         9   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Nov  2 2022  1:34PM  249908        10    ISDIN Corporation         8   
1  Nov  2 2022  1:31PM  249907        19  AdvaGen Pharma, Ltd        12   
2  Nov  2 2022  1:30PM  249906        21    NBTY Global, Inc.         1   
3  Nov  2 2022  1:18PM  249905        10       ClearSpec, LLC         1   
4  Nov  2 2022  1:16PM  249904        10    ISDIN Corporation         9   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4247902        72       52.0       11.0
15          499693         6       13.0       12.0
16          499772         2        0.0        0.0
19          499806        14        0.0        0.0
20          249900         1        0.0        0.0
21         2748798        11        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4247902        72       52.0       11.0
1        15   499693         6       13.0       12.0
2        16   499772         2        0.0        0.0
3        19   499806        14        0.0        0.0
4        20   249900         1        0.0        0.0
5        21  2748798        11        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        72       52.0       11.0
1        15         6       13.0       12.0
2        16         2        0.0        0.0
3        19        14        0.0        0.0
4        20         1        0.0        0.0
5        21        11        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   72.0
1        15  Released    6.0
2        16  Released    2.0
3        19  Released   14.0
4        20  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16    19   20    21
Status                                     
Completed  11.0  12.0  0.0   0.0  0.0   0.0
Executing  52.0  13.0  0.0   0.0  0.0   0.0
Released   72.0   6.0  2.0  14.0  1.0  11.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16    19   20    21
0          Completed  11.0  12.0  0.0   0.0  0.0   0.0
1          Executing  52.0  13.0  0.0   0.0  0.0   0.0
2           Released  72.0   6.0  2.0  14.0  1.0  11.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16    19   20    21
0  Completed  11.0  12.0  0.0   0.0  0.0   0.0
1  Executing  52.0  13.0  0.0   0.0  0.0   0.0
2   Released  72.0   6.0  2.0  14.0  1.0  11.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()